In [ ]:
import pandas as pd

In [ ]:
# load data
data = pd.read_csv('data/SMID_norms.csv')

In [ ]:
# get list of all columns
cols = data.columns
print(cols)

In [ ]:
moral_cols = ['img_name', 'moral_mean', 'moral_sd', 'moral_se', 'moral_n', 'prop_immoral', 'prop_moral', 'prop_neutral']
moral_data = data[moral_cols]

In [ ]:
moral_data.head()

In [ ]:
# add is_moral column, if max of ('prop_immoral', 'prop_moral', 'prop_neutral') is 'prop_moral' set it to 1
# if max of ('prop_immoral', 'prop_moral', 'prop_neutral') is 'prop_immoral' set it to -1
# if max of ('prop_immoral', 'prop_moral', 'prop_neutral') is 'prop_neutral' set it to 0
moral_data.loc[:, 'is_moral'] = moral_data[['prop_immoral', 'prop_moral', 'prop_neutral']].idxmax(axis=1).apply(lambda x: 1 if x == 'prop_moral' else -1 if x == 'prop_immoral' else 0)

In [ ]:
moral_data.head()

In [ ]:
moral_data.tail()

In [ ]:
# load responses_llava.csv
# responses = pd.read_csv('data/kosmos_to_llama_250.csv')
responses = pd.read_csv('data/responses_llava.csv')

In [ ]:
responses.columns

In [ ]:
import re

def find_integer_in_string(s):
    numbers = re.findall(r'\b\d+\b', s)
    return int(numbers[0]) if numbers else None

In [ ]:
# cast responses['generated_text_two'] to int into a new column called llm_rating, save the indices of the rows that fail to cast as int
failed_cast = []
for i, x in enumerate(responses['llama_ratings']):
    try:
        responses.loc[i, 'llm_ratings'] = int(x)
    except ValueError:
        failed_cast.append(i)
        # find the integer in the first sentence of the response
        try:
            first_sentence = x.split('.')[0]
            responses.loc[i, 'llm_ratings'] = find_integer_in_string(first_sentence)
        except ValueError:
            responses.loc[i, 'llm_ratings'] = None

In [ ]:
responses['llama_ratings'].iloc[failed_cast]

In [ ]:
len(failed_cast)

In [ ]:
for i in failed_cast:
    print(responses.loc[i, 'llama_ratings'])

In [ ]:
# check if there are any NaN values in llm_rating
responses['llm_ratings'].isna().sum()

In [ ]:
# add is_moral_llm column to responses dataframe, if generated_text_two is > 3 set it to 1, if generated_text_two is < 3 set it to -1, if generated_text_two is 3 set it to 0
responses.loc[:, 'is_moral_llm'] = responses['llm_ratings'].apply(lambda x: 1 if x > 3 else -1 if x < 3 else 0)

In [ ]:
# cast responses['generated_text_three] to int into a new column called vlm_rating, save the indices of the rows that fail to cast as int
failed_cast = []
for i, x in enumerate(responses['generated_text_three']):
    try:
        responses.loc[i, 'vlm_rating'] = int(x)
    except ValueError:
        failed_cast.append(i)

In [ ]:
failed_cast

In [ ]:
# add is_moral_vlm column to responses dataframe, if vlm_rating is > 3 set it to 1, if vlm_rating is < 3 set it to -1, if vlm_rating is 3 set it to 0
responses.loc[:, 'is_moral_vlm'] = responses['vlm_rating'].apply(lambda x: 1 if x > 3 else -1 if x < 3 else 0)

In [ ]:
responses.head()

In [ ]:
responses.tail()

In [ ]:
# find immoral responses
immoral_responses = responses[responses['is_moral_llm'] == -1]

In [ ]:
immoral_responses.head()

In [ ]:
immoral_responses = responses[responses['is_moral_vlm'] == -1]

In [ ]:
immoral_responses

In [ ]:
moral_data['is_moral'].index

In [ ]:
responses['is_moral_llm'].index

In [ ]:
# remove data/img/ from image names in responses dataframe
responses['image_names'] = responses['image_names'].apply(lambda x: x.replace('data/img/', ''))

# remove suffix (.jpg, ...) from image names in responses dataframe
responses['image_names'] = responses['image_names'].apply(lambda x: x.split('.')[0])

In [ ]:
responses.head()

In [ ]:
moral_data.loc[moral_data['img_name'] == 'b999_p491_9', 'is_moral'].values[0]

In [ ]:
# we want two things:
# (1) Percent Correct: This one shows how often the model's moral judgment matches the majority of the human participants' moral judgments, and
# (2) a root-mean-square error (RMSE) plot, which indicates the RMSE between the model's moral judgement and the average score reported by the human participants. 

# (1) Percent Correct
# moral_data['is_moral'] is the majority human judgment
# responses['is_moral_llm'] is the llm model judgment
# responses['is_moral_vlm'] is the vlm model judgment

# moral_data['img_name] is the image name
# responses['image_names'] is the image name

# calculate the number of correct moral judgments for llm model for all images
correct_llm = 0
for i, x in enumerate(responses['image_names']):
    if moral_data.loc[moral_data['img_name'] == x, 'is_moral'].values[0] == responses.loc[i, 'is_moral_llm']:
        correct_llm += 1

total_llm = len(responses['is_moral_llm'])
percent_correct_llm = correct_llm / total_llm * 100
percent_correct_llm

In [ ]:
# calculate the number of correct moral judgments for vlm model for all images
correct_vlm = 0
for i, x in enumerate(responses['image_names']):
    if moral_data.loc[moral_data['img_name'] == x, 'is_moral'].values[0] == responses.loc[i, 'is_moral_vlm']:
        correct_vlm += 1

total_vlm = len(responses['is_moral_vlm'])
percent_correct_vlm = correct_vlm / total_vlm * 100
percent_correct_vlm

In [ ]:
left = moral_data.loc[moral_data['img_name'].isin(responses['image_names']), 'img_name']
right = responses.loc[responses['image_names'].isin(moral_data['img_name']), 'image_names']

assert left.isin(right).all() and right.isin(left).all(), "The 'img_name' in moral_data and 'image_names' in responses do not match"

In [ ]:
# (2) RMSE
# moral_data['moral_mean'] is the average score reported by the human participants
# responses['llm_rating'] is the llm model judgment
# responses['vlm_rating'] is the vlm model judgment

# moral_data['img_name] is the image name
# responses['image_names'] is the image name

from sklearn.metrics import mean_squared_error
import numpy as np

# calculate the RMSE between the llm model judgment and the average score reported by the human participants for all matching images
merged_data = pd.merge(moral_data, responses, left_on='img_name', right_on='image_names')
llm_mse = mean_squared_error(merged_data['moral_mean'], merged_data['llm_ratings'])
llm_mse

In [ ]:
llm_rmse = np.sqrt(llm_mse)
llm_rmse

In [ ]:
vlm_mse = mean_squared_error(merged_data['moral_mean'], merged_data['vlm_rating'])
vlm_mse

In [ ]:
vlm_rmse = np.sqrt(vlm_mse)
vlm_rmse

In [ ]:
moral_data

In [ ]:
# find all rows where moral_data['is_moral'] == -1
immoral_data = moral_data[moral_data['is_moral'] == -1]

In [ ]:
immoral_data

In [ ]:
# choose the the rows where prop_immoral is the above 0.8
immoral_data = immoral_data[immoral_data['prop_immoral'] > 0.8]

In [ ]:
immoral_data

In [ ]:
#responses['is_moral_llm'] is the llm model judgment

In [ ]:
kosmos_raw = pd.read_csv('data/results_kosmos-2-patch14-224.csv')

In [ ]:
kosmos_raw.columns

In [ ]:
kosmos_raw['generated_text_three'].unique()